In [4]:
import pandas as pd
import sqlite3

# Paths to your CSV and Excel files
csv_file_path = '2020.csv'  
excel_file_path = 'Data_Mapping.xlsx'

# Load CSV data into a Pandas DataFrame
data_2020 = pd.read_csv(csv_file_path)

# Load Excel file containing the mapping (Entity -> Tables, Attribute -> Columns)
mapping_df = pd.read_excel(excel_file_path)

# Open SQLite connection
conn = sqlite3.connect('behavioral_risk_2020.db')

# Iterate over unique entities (table names) from the Excel file
for entity in mapping_df['Entity'].unique():
    # Get the columns (attributes) related to this entity (table)
    entity_columns = mapping_df[mapping_df['Entity'] == entity]['Attribute'].tolist()
    
    # Filter the entity columns that are available in the CSV data
    available_columns = [col for col in entity_columns if col in data_2020.columns]
    
    if available_columns:  # Proceed only if there are available columns
        # Subset the CSV data to include only these columns
        entity_data = data_2020[available_columns]
        
        # Store the subset data into a table in SQLite with the name of the entity
        entity_data.to_sql(entity, conn, if_exists='replace', index=False)
        print(f"Table '{entity}' created successfully with columns: {available_columns}")
    else:
        print(f"No matching columns found in the CSV for table '{entity}'.")

# Close the SQLite connection
conn.close()

print("All data successfully loaded into corresponding tables in SQLite.")


Table 'Alcohol' created successfully with columns: ['AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS']
Table 'Cancer_test' created successfully with columns: ['HADHYST2', 'CNCRDIFF', 'CNCRAGE', 'CSRVTRT3', 'CSRVDOC1', 'CSRVSUM', 'CSRVRTRN', 'CSRVINST', 'CSRVINSR', 'CSRVDEIN', 'CSRVCLIN', 'CSRVPAIN', 'CSRVCTL2', 'PSATEST1']
Table 'Caregiver' created successfully with columns: ['CAREGIV1', 'CRGVREL4', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD', 'CRGVPER1', 'CRGVHOU1', 'CRGVEXPT']
Table 'childhood_exp' created successfully with columns: ['ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM', 'ACEHVSEX']
Table 'Children_details' created successfully with columns: ['RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX']
Table 'Cognitive_Decline' created successfully with columns: ['CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'CDDISCUS']
Table 'Common_Variables ' created successfully with columns: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH